In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Critere de performance
def compute_pred_score(y_true, y_pred):
    y_pred_unq =  np.unique(y_pred)
    for i in y_pred_unq:
        if((i != -1) & (i!= 1) & (i!= 0) ):
            raise ValueError('The predictions can contain only -1, 1, or 0!')
    y_comp = y_true * y_pred
    score = float(10*np.sum(y_comp == -1) + np.sum(y_comp == 0))
    score /= y_comp.shape[0]
    return score

In [3]:
X_train_fname = 'training_templates.csv'
y_train_fname = 'training_labels.txt'
X_test_fname  = 'testing_templates.csv'
X_train = pd.read_csv(X_train_fname, sep=',', header=None).values
X_test  = pd.read_csv(X_test_fname,  sep=',', header=None).values
y_train = np.loadtxt(y_train_fname, dtype=np.int)

### 1) Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from time import time

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

""" Grid Search
tuned_parameters = [{'penalty': ['l1'], 'C': np.logspace(-1, 2, 20), 'solver': ['liblinear']},
                    {'penalty': ['l2'], 'C': np.logspace(-1, 2, 20), 'solver': ['sag']}]

clf = GridSearchCV(LogisticRegression(), tuned_parameters, cv=5)

t0 = time()  
clf.fit(X_train_scaled, y_train)
print("done in %0.3fs" % (time() - t0))

print("Best score: %0.3f" % clf.best_score_)
"""
t0 = time()
print "Use l1 penalty:"
clf1 = LogisticRegressionCV(penalty='l1', Cs=20, solver='liblinear', cv=5, refit=True, scoring='roc_auc')
clf1.fit(X_train_scaled, y_train)
y_pred_train1 = clf1.predict(X_train_scaled)
score = compute_pred_score(y_train, y_pred_train1)
print("done in %0.3fs" % (time() - t0))
print('Score sur le train : %s' % score)

print "Use l2 penalty:"
t0 = time()
clf2 = LogisticRegressionCV(penalty='l2', Cs=20, solver='sag', cv=5, refit=True, scoring='roc_auc')
clf2.fit(X_train_scaled, y_train)
y_pred_train2 = clf2.predict(X_train_scaled)
score = compute_pred_score(y_train, y_pred_train2)
print("done in %0.3fs" % (time() - t0))
print('Score sur le train : %s' % score)

"""
# Fit model
clf = LogisticRegression()
clf.fit(X_train_scaled, y_train)

# Prediction
y_pred_train =  clf.predict(X_train_scaled)

# Compute the score
score = compute_pred_score(y_train, y_pred_train)
print('Score sur le train : %s' % score)
"""

Use l1 penalty:
done in 204.913s
Score sur le train : 0.635700757576
Use l2 penalty:
done in 173.663s
Score sur le train : 0.635511363636


"\n# Fit model\nclf = LogisticRegression()\nclf.fit(X_train_scaled, y_train)\n\n# Prediction\ny_pred_train =  clf.predict(X_train_scaled)\n\n# Compute the score\nscore = compute_pred_score(y_train, y_pred_train)\nprint('Score sur le train : %s' % score)\n"

In [6]:
y_pred = clf.predict(X_test)
np.savetxt('y_pred.txt', y_pred, fmt='%d')